Why is this interesting?

Depending on how we caluclate gradients, some things are learnable while others are not. 
Linear networks (trained with SGD) have been known not to ...
Given a better teacher, it is possible to ... 



The difference between model-based and model-free optimisation. 
The model-based optimisation (SGD -- where we use a linear model, ie the gradient) is very close to being accurate, but is not quite true. VS. Model-free optimisation doesnt care, it just looks at outputs and ...

In [1]:
import numpy as np
import numpy.random as rnd

import torch as tr
import sklearn.datasets

In [7]:
dtype = tr.uint8
n_pop = 128
hidden_size = 64
batch_size = 512
n_hidden_layers = 1

In [8]:
ds = sklearn.datasets.load_digits(n_class=10, return_X_y=False)
print(ds.images.shape, ds.images.dtype)
images = tf.reshape(tf.cast(ds.images, dtype), (-1, 64))
labels = tf.cast(ds.target, tf.int64)  # tf.one_hot(ds.target, 10)

(1797, 8, 8) float64


In [9]:
def func(x, l, params, get_acc=False):
    # TODO. tensorflow doesnt have a lowdim matmul available in the python api...
    # https://github.com/google/gemmlowp
    h = tf.matmul(x, params['weights_first']) + params['biases_first']
    for i in range(n_hidden_layers):
        h = tf.matmul(h, params['weights_{}'.format(i)]) + params['biases_{}'.format(i)]
    y = tf.matmul(h, params['weights_last']) + params['biases_last']

    p = tf.nn.softmax(y)
    if get_acc:
        return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(p, axis=1), l), 
                             tf.float32))
    else:
        return tf.losses.sparse_softmax_cross_entropy(labels=l,logits=y)
    
params = {'weights_first': tf.random_normal((64, hidden_size), stddev=np.sqrt(2./64), dtype=dtype), 
          'biases_first': tf.random_normal((1, hidden_size), stddev=np.sqrt(2./64), dtype=dtype),
          'weights_last': tf.random_normal((hidden_size, 10), stddev=np.sqrt(2./hidden_size), dtype=dtype), 
          'biases_last': tf.random_normal((1, 10), stddev=np.sqrt(2./hidden_size), dtype=dtype)}
weights = {'weights_{}'.format(i): tf.random_normal((hidden_size, hidden_size), 
                                                    stddev=np.sqrt(2./hidden_size), 
                                                    dtype=dtype)
          for i in range(n_hidden_layers)}
biases = {'biases_{}'.format(i): tf.random_normal((1, hidden_size), 
                                                    stddev=np.sqrt(2./hidden_size), 
                                                    dtype=dtype)
         for i in range(n_hidden_layers)}
params = dict(params, **weights)
params = dict(params, **biases)

# def create(shape, dtype):
#     return tf.cast(tf.random_uniform(shape, minval=-2**7, maxval=2**7), dtype)

# params = {'weights1':create((64, hidden_size), dtype), 
#           'biases1': create((1, hidden_size), dtype),
#           'weights2': create((hidden_size, 10), dtype), 
#           'biases2': create((1, 10), dtype)}

o = func(images, labels, params)

In [10]:
def project(M, v):
    shape = M.get_shape()
    M = tf.reshape(M, (tf.shape(M)[0], -1))
    sim = tf.losses.cosine_distance(tf.nn.l2_normalize(x, 0), tf.nn.l2_normalize(y, 0), dim=0)
    M *= sim
    return tf.reshape(M, shape)

def create_noise(params, N, momentum, stddev):
    return {k: tf.random_normal((N, v.shape[0], v.shape[1]), 
                                dtype=dtype, 
                                stddev=stddev)
           for k, v in params.items()}

def mutate_and_evaluate(func, params, images, labels, n_pop, n_iters, step, momentum):
    noise = create_noise(params, n_pop, momentum, stddev=0.1*(1-step/n_iters))
    losses = []

    # for each elem in the population
    for i in range(n_pop):  # this should be parallelised!
        current_params = {k: params[k]+v[i, ...]
                         for k, v in noise.items()}

        # do we really need to evaluate on all the data?
        idx = np.random.randint(0, images.shape[0]-batch_size)
        # if I am going to do this I really need some mometum
        # as this algol has no memory. so good results are forgotten

        # use our fittness fn/loss fn to evaluate
        loss = func(images[idx: idx+batch_size, ...], 
                    labels[idx: idx+batch_size, ...], 
                    current_params)
        losses.append(loss)
    #     print({k: v.shape for k,v in current_params.items()})

    losses = tf.reshape(tf.stack(losses), (1, n_pop))
    return noise, losses

def update_params(losses, noise, params, 
                  lr=1.0, weighted_sum=True):
    new_params = {}
    m, v = tf.nn.moments(losses, 1)
    losses = (losses - m)/v

    momentum = {}
    for k in params.keys():
        # take a weighted sum
        if weighted_sum:
            dp = tf.reshape(tf.matmul(losses, 
                                      tf.reshape(noise[k], 
                                                (n_pop, -1))),
                            params[k].shape)
        # or could just pick the best. argmax.
        else:
            # this one seems to work better!?
            dp = -noise[k][tf.argmin(losses, axis=1)[0]]
            # elif:
            # would also like to be able to sub in gradients from AD
        
        momentum[k] = dp

#         print(np.sum(np.abs(dp)))
        # do gradient descent on the params
        new_params[k] = params[k] - lr*dp

    return new_params, momentum

In [ ]:
class Optimiser():
    def __init__(self):
        pass

class ES(Optimiser):
    pass

class SGD(Optimiser):
    pass
    

In [11]:
iters=1000
momentum = {k: tf.zeros_like(v) for k,v in params.items()}
for i in range(iters):
    noise, losses = mutate_and_evaluate(func, params, images, labels, n_pop, iters, i, momentum)
    params, momentum = update_params(losses, noise, params)
#     print(np.mean(losses))
    acc = func(images, labels, params, get_acc=True)
    print('\rstep: {} loss: {}'.format(i, acc), end='', flush=True)

step: 99 loss: 0.8074568510055542

In [12]:
def train(method, N):
    if method is 'es':
        pass
    if method is 'sgd':
        pass
    
    
es_loss = train('es', 3)
sgd_loss = train('sgd', 3)

plot(es_loss)
plot(sgd_loss)
# want to see that; 
# - es is slow
# - sgd achieves lower accuracy (on linear nets)
# - momentum!?

https://blog.openai.com/nonlinear-computation-in-linear-networks/

> As a consequence of these conventions and the binary format used, the smallest normal non-zero number (in binary) is 1.0..0 x 2^-126, which we refer to as min going forward. However, the next representable number is 1.0..01 x 2^-126, which we can write as min + 0.0..01 x 2^-126. It is evident that the gap between the 2nd number is by a factor of 2^20 smaller than gap between 0 and min. In float32, when numbers are smaller than the smallest representable number they get mapped to zero. Due to this ‘underflow’, around zero all computation involving floating point numbers becomes nonlinear.

Ok cool. But how to train it...

https://arxiv.org/abs/1703.03864

https://gist.github.com/karpathy/77fbb6a8dac5395f1b73e7a89300318d

Questions to investigate;

* When does it do better than SGD?
* What if we instead used ES to estimate gradients and bproped them?
* How can we get higher performance? 
    * More non-linearity? 
    * Momentum for ES?
    * ?
* What about overflow? What about `tf.float16`s or `tf.bfloat16` (16-bit truncated floating-point) or `tf.qint8` (Quantized 8-bit signed integer) or `tf.uint8`?
* How do estimates of gradients effect generalisation?

While it may be a 'linear' network, it has lost associativity, and cannot be factored back down to a single linear op. Lost interpretability. Effectively just a deep network, with step functions as the activations. 